In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Flatten, Conv2D, Conv1D
from keras.layers import LSTM, Reshape
from keras.layers import GRU
from keras.callbacks import ModelCheckpoint,EarlyStopping
import tensorflow as tf

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="3"
gpus = tf.config.experimental.list_logical_devices('GPU')
print(gpus)
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except:
        print('checking gpu error')
print('checking GPUs')

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config=tf.compat.v1.ConfigProto()
# config.gpu_options.visible_device_list= '0,1'
config.gpu_options.allow_growth=True
config.log_device_placement=True
# config.visible_device_list =2
sess=tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

[LogicalDevice(name='/device:GPU:0', device_type='GPU')]
checking gpu error
checking GPUs
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: TITAN X (Pascal), pci bus id: 0000:0a:00.0, compute capability: 6.1



In [33]:
def _build_model(n_actions):
    model_sub_in = Input(shape=(1, 7))
    # goalx,goaly, posx, posy, v, a, theta
    model_sub_flatten = Flatten()(model_sub_in)
    model_sub_mid = Dense(128, activation='relu', name='layer_sub_mid')(model_sub_flatten)
    model_sub_out = Dense(256, activation='elu', name='layer_sub_out')(model_sub_mid)
    model_sub = Model(model_sub_in, model_sub_out)

    model_veh_in = Input(shape=(5, 10, 2))
    # history traj veh
    model_veh_d = Dense(1,  activation='relu', name='layer_veh_dx')(model_veh_in)
    veh_reshape = Reshape((-1,10))(model_veh_d)
    model_veh_lstm = LSTM(256)(veh_reshape)
    veh_reshape2 = Reshape((3,8,8))(model_veh_lstm)
    
    model_veh_conv1 = Conv2D(128,(2,2),  activation='relu', name='layer_veh_conv1')(veh_reshape2)
    model_veh_conv2 = Conv2D(256,(2,2),  activation='relu', name='layer_veh_conv2')(model_veh_conv1)
    model_veh_flatten = Flatten()(model_veh_conv2)
    model_veh_out = Dense(128,  activation='relu', name='layer_veh_out')(model_veh_flatten)
    model_veh = Model(model_veh_in, model_veh_out)
    
    model_ped_in = Input(shape=(3, 10, 2))
    # history traj ped
    model_ped_d= Dense(1,  activation='relu', name='layer_ped_dx')(model_ped_in)
    ped_reshape = Reshape((-1,10))(model_ped_d)
    model_ped_lstm = LSTM(256)(ped_reshape)
    ped_reshape2 = Reshape((3,8,8))(model_ped_lstm)
    
    model_ped_conv1 = Conv2D(128,(2,2),  activation='relu', name='layer_ped_conv1')(ped_reshape2)
    model_ped_conv2 = Conv2D(256,(2,2),  activation='relu', name='layer_ped_conv2')(model_ped_conv1)
    model_ped_flatten = Flatten()(model_ped_conv2)
    model_ped_out = Dense(128,  activation='elu', name='layer_ped_out')(model_ped_flatten)
    model_ped = Model(model_ped_in, model_ped_out)
    
    
    model_cyc_in = Input(shape=(3, 10, 2))
    # history traj cyc
    model_cyc_d= Dense(1,  activation='relu', name='layer_cyc_dx')(model_cyc_in)
    cyc_reshape = Reshape((-1,10))(model_cyc_d)
    model_cyc_lstm = LSTM(256)(cyc_reshape)
    cyc_reshape2 = Reshape((3,8,8))(model_cyc_lstm)
    
    model_cyc_conv1 = Conv2D(128,(2,2),  activation='relu', name='layer_cyc_conv1')(cyc_reshape2)
    model_cyc_conv2 = Conv2D(128,(2,2),  activation='relu', name='layer_cyc_conv2')(model_cyc_conv1)
    model_cyc_flatten = Flatten()(model_cyc_conv2)
    model_cyc_out = Dense(128,  activation='elu', name='layer_cyc_out')(model_cyc_flatten)
    model_cyc = Model(model_cyc_in, model_cyc_out)
    
    
    
    model_edge_in = Input(shape=(1, 11))
    # edges connected to subject
    model_edge_flatten = Flatten()(model_edge_in)
    model_edge_mid = Dense(128,  activation='relu', name='layer_edge_mid')(model_edge_flatten)
    model_edge_out = Dense(256,  activation='relu', name='layer_edge_output')(model_edge_mid)
    model_edge = Model(model_edge_in, model_edge_out)


    concatenated = concatenate([model_sub_out, model_veh_out,model_ped_out,model_cyc_out,model_edge_out])
    graph_out = Dense(512, activation='softmax', name='graph_out_layer')(concatenated)
    
    # define the deep RL model below.
    rl0 = Dense(512,  activation='relu', name='layer_rl_0')(graph_out)
    rl1 = Dense(1024,  activation='relu', name='layer_rl_1')(rl0)
    rl2 = Dense(1024,  activation='relu', name='layer_rl_2')(rl1)
    out = Dense(n_actions,  activation='relu', name='layer_q_values')(rl2)
    
    # action space 
#     print(len(np.arange(-3,3,0.1)))
#     print(len(np.arange(0,360,5)))
    
    merged_model = Model([model_sub_in, model_veh_in,model_ped_in,model_cyc_in,model_edge_in], out)
    return merged_model


In [34]:
merged_model = _build_model(n_actions=4320)
merged_model.compile(loss='binary_crossentropy', optimizer='adagrad', 
metrics=['accuracy'])
merged_model.summary()

checkpoint = ModelCheckpoint('weights.h5', monitor='val_acc',
                             save_best_only=True, verbose=2)
early_stopping = EarlyStopping(monitor="val_loss", patience=10)


Model: "model_61"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_51 (InputLayer)           [(None, 5, 10, 2)]   0                                            
__________________________________________________________________________________________________
input_52 (InputLayer)           [(None, 3, 10, 2)]   0                                            
__________________________________________________________________________________________________
input_53 (InputLayer)           [(None, 3, 10, 2)]   0                                            
__________________________________________________________________________________________________
layer_veh_dx (Dense)            (None, 5, 10, 1)     3           input_51[0][0]                   
___________________________________________________________________________________________

In [78]:
#synthetic data
sub = np.random.rand(320000, 1,7)
veh =np.random.rand(320000,3,10,2)
ped =np.random.rand(320000,3,10,2)
cyc =np.random.rand(320000,3,10,2)
edge =np.random.rand(320000,1,9)
y=np.random.randint(0,20000,size=1800)
edge.shape

(320000, 1, 9)

In [79]:

merged_model.fit([sub, veh, ped, cyc, edge], y=y, batch_size=32, epochs=10,
             verbose=1, validation_split=0.1, shuffle=True
# callbacks=[early_stopping, checkpoint]
                )

Epoch 1/10
9000/9000 [==============================] - 53s 6ms/step - loss: 460.6199 - accuracy: 1.0417e-05 - val_loss: 5.5560e-04 - val_accuracy: 0.0000e+00
Epoch 2/10
9000/9000 [==============================] - 53s 6ms/step - loss: 460.6173 - accuracy: 0.0000e+00 - val_loss: 5.5560e-04 - val_accuracy: 0.0000e+00
Epoch 3/10
9000/9000 [==============================] - 53s 6ms/step - loss: 460.6171 - accuracy: 6.9444e-06 - val_loss: 5.5560e-04 - val_accuracy: 0.0000e+00
Epoch 4/10
9000/9000 [==============================] - 53s 6ms/step - loss: 460.6169 - accuracy: 3.4722e-06 - val_loss: 5.5560e-04 - val_accuracy: 0.0000e+00
Epoch 5/10
9000/9000 [==============================] - 53s 6ms/step - loss: 460.6168 - accuracy: 2.4306e-05 - val_loss: 5.5560e-04 - val_accuracy: 0.0000e+00
Epoch 6/10
9000/9000 [==============================] - 53s 6ms/step - loss: 460.6168 - accuracy: 1.3889e-05 - val_loss: 5.5560e-04 - val_accuracy: 0.0000e+00
Epoch 7/10
9000/9000 [========================